In [1]:
import os
import numpy as np
import pandas as pd
from scipy import signal
import pickle
from util import *


data = []
for number in range(1, 7):
    
    # read data file
    name = 'sula_' + "%02d" % number + '.xls'
    filename = '../rawdata/sula/' + name
    xls_data = pd.read_excel(filename, header=1)        
    
    dat = xls_data[['long1', 'lat1', 'speed1', 'long2', 'lat2', 'speed2']]
    
    if number == 3 or number == 4 or number == 5:
        dat = xls_data[['long1', 'lat1', 'speed1', 'long2', 'lat2', 'speed2', 'long3', 'lat3', 'speed3']]
    
    dat = dat.replace('-', np.nan)
    dat = dat.interpolate(limit_direction='both')

    px1 = np.array(dat['long1'])
    py1 = np.array(dat['lat1'])
    px2 = np.array(dat['long2'])
    py2 = np.array(dat['lat2'])
    
    if number == 3 or number == 4 or number == 5:
        px3 = np.array(dat['long3']) 
        py3 = np.array(dat['lat3'])

    # degree to meter
    px1 = deg2meter(px1)
    py1 = deg2meter(py1)
    px2 = deg2meter(px2)
    py2 = deg2meter(py2)
    
    if number == 3 or number == 4 or number == 5:
        px3 = deg2meter(px3) 
        py3 = deg2meter(py3)

    # low-pass filter
    samplerate, fp, fs, gpass, gstop = 1, 0.02, 0.3, 1, 2
    px1 = lowpass(px1, samplerate, fp, fs, gpass, gstop)
    py1 = lowpass(py1, samplerate, fp, fs, gpass, gstop)
    px2 = lowpass(px2, samplerate, fp, fs, gpass, gstop)
    py2 = lowpass(py2, samplerate, fp, fs, gpass, gstop)
    
    if number == 3 or number == 4 or number == 5:
        px3 = lowpass(px3, samplerate, fp, fs, gpass, gstop) 
        py3 = lowpass(py3, samplerate, fp, fs, gpass, gstop)

    # shift to origin
    x_min = np.min([np.min(px1), np.min(px2)])
    y_min = np.min([np.min(py1), np.min(py2)])
    
    if number == 3 or number == 4 or number == 5:
        x_min = np.min([np.min(px1), np.min(px2), np.min(px3)])
        y_min = np.min([np.min(py1), np.min(py2), np.min(py3)])

    px1 -= x_min
    py1 -= y_min
    px2 -= x_min 
    py2 -= y_min
    
    if number == 3 or number == 4 or number == 5:
        px3 -= x_min 
        py3 -= y_min

    # calculate velocity (difference)
    dpx1 = np.diff(px1)
    dpy1 = np.diff(py1)
    dpx2 = np.diff(px2)
    dpy2 = np.diff(py2)
    
    if number == 3 or number == 4 or number == 5:
        dpx3 = np.diff(px3)
        dpy3 = np.diff(py3)

    # ajust position data length to velocity data
    px1 = px1[:-1]
    py1 = py1[:-1]
    px2 = px2[:-1]
    py2 = py2[:-1]
    
    if number == 3 or number == 4 or number == 5:
        px3 = px3[:-1]
        py3 = py3[:-1]

    # get speed
    s1 = np.array(dat['speed1'])
    s2 = np.array(dat['speed2'])
    
    if number == 3 or number == 4 or number == 5:
        s3 = np.array(dat['speed3'])

    # append data
    data1 = []
    data1.append(dpx1); data1.append(dpy1);
    data1.append(px1); data1.append(py1);
    data1.append(s1)

    data2 = []
    data2.append(dpx2); data2.append(dpy2);
    data2.append(px2); data2.append(py2);
    data2.append(s2)

    if number == 3 or number == 4 or number == 5:
        data3 = []
        data3.append(dpx3); data3.append(dpy3);
        data3.append(px3); data3.append(py3);
        data3.append(s3)        

    data_seq = []
    data_seq.append(data1)
    data_seq.append(data2)
    
    if number == 3 or number == 4 or number == 5:
        data_seq.append(data3)

    data.append(data_seq)
        
np.save('../npy/sula_data', data)